### Calculate installed capacity at offshore area vs max possible

In [87]:
import pandas as pd
from constants import WIND_FARM_NODES

pd.options.mode.chained_assignment = None  # default='warn'

CASES = ["Base", "NOgrid", "NoHubs", "NoHubsNOgrid", "OnlyCentral", "OnlyEU", "OnlyNorth"]
DF_BY_CASES_SPLIT = dict({})
DF_BY_CASES_COMBO = dict({})

GEN_TYPE_MAPPING = dict({
    "Wind_offshr_grounded": "Windoffshoregrounded",
    "Wind_offshr_floating": "Windoffshorefloating",
})

max_capacaities = pd.read_csv("../EMPIRE_extension/Output/offshore_capacities_base.csv")\
                        .rename(columns={"GeneratorTechnology": "GeneratorType", "generatorMaxInstallCapacity in MW": "Max capacity [MW]"})
max_capacaities["Node"] = max_capacaities["Node"].apply(lambda n: n.replace(" ", ""))
max_capacaities["GeneratorType"] = max_capacaities["GeneratorType"].apply(lambda g: GEN_TYPE_MAPPING[g])
max_capacaities = max_capacaities.drop(columns=["area_km2"])
max_capacaities

,Node,GeneratorType,Max capacity [MW]
0,MorayFirth,Windoffshoregrounded,5465.000000
1,FirthofForth,Windoffshoregrounded,10530.000000
2,DoggerBank,Windoffshoregrounded,16260.000000
3,Hornsea,Windoffshoregrounded,12055.000000
4,OuterDowsing,Windoffshoregrounded,6265.000000
5,Norfolk,Windoffshoregrounded,8130.000000
6,EastAnglia,Windoffshoregrounded,6755.000000
7,Borssele,Windoffshoregrounded,4881.916667
8,HollandseeKust,Windoffshoregrounded,6664.166667
9,HelgoländerBucht,Windoffshoregrounded,22910.625000


In [88]:
for case in CASES:
    _df = pd.read_csv(f'Results/{case}/results_output_gen.csv')

    # Offshore nodes
    df_wind_farm_nodes_combo = _df[_df['Node'].isin(WIND_FARM_NODES)]
    df_wind_farm_nodes_split= _df[_df['Node'].isin(WIND_FARM_NODES)]

    # Sum floating + grounded cap (combo)
    df_wind_farm_nodes_combo["genInstalledCap_MW"] = df_wind_farm_nodes_combo.groupby(['Node', 'Period'])['genInstalledCap_MW'].transform('sum')
    df_wind_farm_nodes_combo = df_wind_farm_nodes_combo.drop_duplicates(("Node", "Period"), ignore_index=True)\
                                [["Node", "Period", "genInstalledCap_MW"]]
    df_wind_farm_nodes_combo = df_wind_farm_nodes_combo.merge(max_capacaities, on=["Node"], how="left")\
                                .drop(columns=["GeneratorType"]).drop_duplicates(["Node", "Period"])
    df_wind_farm_nodes_combo["Installed capacity of max (%)"] = round(df_wind_farm_nodes_combo["genInstalledCap_MW"] \
                                                                        / df_wind_farm_nodes_combo["Max capacity [MW]"] * 100, 1)
    DF_BY_CASES_COMBO[case] = df_wind_farm_nodes_combo

    # Floating + grounded splitted
    df_wind_farm_nodes_split["genInstalledCap_MW"] = df_wind_farm_nodes_split.groupby(['Node', 'Period', 'GeneratorType'])\
                                ['genInstalledCap_MW'].transform('sum')
    df_wind_farm_nodes_split = df_wind_farm_nodes_split.drop_duplicates(("Node", "Period", 'GeneratorType'), ignore_index=True)\
                                [["Node", "Period", 'GeneratorType', "genInstalledCap_MW"]]
    df_wind_farm_nodes_split = df_wind_farm_nodes_split.merge(max_capacaities, on=["Node", 'GeneratorType'], how="left")
    df_wind_farm_nodes_split["Installed capacity of max (%)"] = round(df_wind_farm_nodes_split["genInstalledCap_MW"] \
                                                                        / df_wind_farm_nodes_split["Max capacity [MW]"] * 100, 1)
    DF_BY_CASES_SPLIT[case] = df_wind_farm_nodes_split

In [91]:
FILTER_CASE = "Base"
FILTER_PERIOD = "2045-2050"
COMBO = True # If False "split" by floating/grounded; else total cap in area

filtered_df = DF_BY_CASES_COMBO[FILTER_CASE] if COMBO else DF_BY_CASES_SPLIT[FILTER_CASE]
filtered_df = filtered_df[(filtered_df["Period"] == FILTER_PERIOD) & filtered_df["Max capacity [MW]"] > 0]
filtered_df["genInstalledCap_MW"] = round(filtered_df["genInstalledCap_MW"])
filtered_df["Max capacity [MW]"] = round(filtered_df["Max capacity [MW]"])
filtered_df = filtered_df.sort_values(by="genInstalledCap_MW", ascending=False).reset_index(drop=True)
filtered_df

,Node,Period,genInstalledCap_MW,Max capacity [MW],Installed capacity of max (%)
0,HelgoländerBucht,2045-2050,22911.0,22911.0,100.0
1,Nordsøen,2045-2050,20110.0,20110.0,100.0
2,DoggerBank,2045-2050,16260.0,16260.0,100.0
3,SørvestF,2045-2050,13510.0,13510.0,100.0
4,Hornsea,2045-2050,12055.0,12055.0,100.0
5,SørvestB,2045-2050,10895.0,10895.0,100.0
6,FirthofForth,2045-2050,10530.0,10530.0,100.0
7,NordvestC,2045-2050,10214.0,27910.0,36.6
8,SørvestE,2045-2050,10160.0,5080.0,200.0
9,SønnavindA,2045-2050,10006.0,14500.0,69.0
